# Clustering Crypto

In [1]:
# Initial imports
import requests
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

### Fetching Cryptocurrency Data

In [3]:
from pathlib import Path
file_path = Path("Resources/crypto_data.csv")

# Create a DataFrame
crypto_df = pd.read_csv(file_path).set_index('Unnamed: 0')
crypto_df.tail()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
Unnamed: 0,,,,,,
XBC,BitcoinPlus,Scrypt,True,PoS,1.283270e+05,1000000
DVTC,DivotyCoin,Scrypt,False,PoW/PoS,2.149121e+07,100000000
GIOT,Giotto Coin,Scrypt,False,PoW/PoS,NaN,233100000
OPSC,OpenSourceCoin,SHA-256,False,PoW/PoS,NaN,21000000
PUNK,SteamPunk,PoS,False,PoS,NaN,40000000


### Data Preprocessing

In [4]:
# Keep only cryptocurrencies that are trading
crypto_df = crypto_df[crypto_df['IsTrading'] ==True]
crypto_df.tail()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
Unnamed: 0,,,,,,
SERO,Super Zero,Ethash,True,PoW,NaN,1000000000
UOS,UOS,SHA-256,True,DPoI,NaN,1000000000
BDX,Beldex,CryptoNight,True,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,True,PoW,7.296538e+06,21000000
XBC,BitcoinPlus,Scrypt,True,PoS,1.283270e+05,1000000


In [5]:
# Remove the "IsTrading" column
del crypto_df['IsTrading']
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
Unnamed: 0,,,,,
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0


In [6]:
# Remove rows with at least 1 null value
crypto_df.dropna(inplace=True)
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
Unnamed: 0,,,,,
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000


In [7]:
# Remove rows with cryptocurrencies having no coins mined
crypto_df = crypto_df[crypto_df.TotalCoinsMined !=0]
crypto_df.tail()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
Unnamed: 0,,,,,
ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000
XBC,BitcoinPlus,Scrypt,PoS,1.283270e+05,1000000


In [8]:
# Drop rows where there are 'N/A' text values
crypto_df.dropna(axis=1)

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
Unnamed: 0,,,,,
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610


In [9]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
coin_name = crypto_df.CoinName
coin_name = pd.DataFrame(coin_name)
coin_name.head()

,CoinName
Unnamed: 0,
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum


In [10]:
crypto_df.drop(['CoinName'], axis =1, inplace=True)
X = pd.get_dummies(crypto_df, columns=['Algorithm', 'ProofType'], drop_first=True)
X.head()

,TotalCoinsMined,TotalCoinSupply,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,Algorithm_Cloverhash,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# Standardize data
X_scaled = StandardScaler().fit_transform(X)
print(X_scaled[0:5])

[[-0.11674788 -0.15286468 -0.0433555  -0.0433555  -0.06137164 -0.07523548
  -0.0433555  -0.06137164 -0.06137164 -0.0433555  -0.0433555  -0.19226279
  -0.06137164 -0.09731237 -0.0433555  -0.11536024 -0.07523548 -0.0433555
  -0.0433555  -0.15176505 -0.0433555  -0.13105561 -0.0433555  -0.0433555
  -0.08695652 -0.0433555  -0.0433555  -0.0433555  -0.0433555  -0.06137164
  -0.0433555  -0.08695652 -0.08695652 -0.08695652 -0.0433555  -0.13105561
  -0.13827675 -0.13827675 -0.0433555  -0.06137164 -0.0433555  -0.07523548
  -0.1815096  -0.0433555  -0.0433555  -0.0433555  -0.07523548 -0.15811388
  -0.3145935  -0.0433555  -0.08695652 -0.07523548 -0.06137164 -0.0433555
   1.38873015 -0.0433555  -0.0433555  -0.06137164 -0.0433555  -0.0433555
  -0.0433555  -0.0433555  -0.0433555  -0.0433555  -0.0433555  -0.0433555
  -0.39836623 -0.0433555  -0.1815096  -0.0433555  -0.08695652 -0.08695652
  -0.10670145 -0.0433555  -0.13105561 -0.0433555  -0.0433555  -0.0433555
  -0.0433555  -0.07523548 -0.4386271  -0.043

### Reducing Dimensions Using PCA

In [12]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)
X_pca = pca.fit_transform(X_scaled)

In [13]:
# Create a DataFrame with the principal components data
X_pca_df = pd.DataFrame(
    data = X_pca, columns=['pc 1', 'pc 2', 'pc 3'],
    index = crypto_df.index
)
X_pca_df.head()

,pc 1,pc 2,pc 3
Unnamed: 0,,,
42,-0.334244,0.978030,-0.607779
404,-0.317566,0.978152,-0.608290
1337,2.309133,1.614950,-0.639284
BTC,-0.144215,-1.359980,0.249451
ETH,-0.152814,-2.045249,0.470917


### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [15]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(X_pca_df)
    inertia.append(km.inertia_)

import altair as alt
# Create the Elbow Curve using altair
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
alt.Chart(df_elbow).mark_circle(size=60).encode(
    x="k",
    y="inertia"
).interactive()

alt.Chart(...)

Running K-Means with `k=4`

In [16]:
# Initialize the K-Means model
model =KMeans(n_clusters=4, random_state=0)

# Fit the model
model.fit(X_pca_df)

# Predict clusters
predictions = model.predict(X_pca_df)

# Create a new DataFrame including predicted clusters and cryptocurrencies features
X_pca_df['class'] = model.labels_
clustered_df = pd.concat([crypto_df, X_pca_df, coin_name], axis=1)
clustered_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,pc 1,pc 2,pc 3,class,CoinName
Unnamed: 0,,,,,,,,,
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.334244,0.978030,-0.607779,1,42 Coin
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.317566,0.978152,-0.608290,1,404Coin
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.309133,1.614950,-0.639284,1,EliteCoin
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.144215,-1.359980,0.249451,0,Bitcoin
ETH,Ethash,PoW,1.076842e+08,0,-0.152814,-2.045249,0.470917,0,Ethereum


# Visualizing Results

#### 3D-Scatter with Clusters

In [17]:
# Create a 2D-Scatter with the PCA data and the clusters
alt.Chart(clustered_df).mark_circle(size=60).encode(
    x='pc 3',
    y='pc 2',
    color='class',
    tooltip=["CoinName", "Algorithm", "TotalCoinsMined", "TotalCoinSupply"]
).interactive()

alt.Chart(...)

#### Table of Tradable Cryptocurrencies

In [17]:
# Print the total number of tradable cryptocurrencies
len(crypto_df.index)

533

#### Scatter Plot with Tradable Cryptocurrencies

In [19]:
# Scale data to create the scatter plot
X = clustered_df[['TotalCoinsMined', 'TotalCoinSupply']]
X_scaled2 = MinMaxScaler().fit_transform(X)
print(X_scaled[0:5])

[[-0.11674788 -0.15286468 -0.0433555  -0.0433555  -0.06137164 -0.07523548
  -0.0433555  -0.06137164 -0.06137164 -0.0433555  -0.0433555  -0.19226279
  -0.06137164 -0.09731237 -0.0433555  -0.11536024 -0.07523548 -0.0433555
  -0.0433555  -0.15176505 -0.0433555  -0.13105561 -0.0433555  -0.0433555
  -0.08695652 -0.0433555  -0.0433555  -0.0433555  -0.0433555  -0.06137164
  -0.0433555  -0.08695652 -0.08695652 -0.08695652 -0.0433555  -0.13105561
  -0.13827675 -0.13827675 -0.0433555  -0.06137164 -0.0433555  -0.07523548
  -0.1815096  -0.0433555  -0.0433555  -0.0433555  -0.07523548 -0.15811388
  -0.3145935  -0.0433555  -0.08695652 -0.07523548 -0.06137164 -0.0433555
   1.38873015 -0.0433555  -0.0433555  -0.06137164 -0.0433555  -0.0433555
  -0.0433555  -0.0433555  -0.0433555  -0.0433555  -0.0433555  -0.0433555
  -0.39836623 -0.0433555  -0.1815096  -0.0433555  -0.08695652 -0.08695652
  -0.10670145 -0.0433555  -0.13105561 -0.0433555  -0.0433555  -0.0433555
  -0.0433555  -0.07523548 -0.4386271  -0.043

In [20]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
X_scaled2_df = pd.DataFrame(X_scaled2)
X_scaled2_df.rename(columns={0: 'TotalCoinsMined', 1: 'TotalCoinSupply'}, inplace=True)
X_scaled2_df['Class'] = model.labels_
X_scaled2_df.head()

,TotalCoinsMined,TotalCoinSupply,Class
0,0.005942,4.200000e-11,1
1,0.007002,5.320000e-04,1
2,0.035342,3.141593e-01,1
3,0.005960,2.100000e-05,0
4,0.006050,0.000000e+00,0


In [21]:
alt.Chart(X_scaled2_df).mark_circle(size=60).encode(
    x="TotalCoinsMined",
    y="TotalCoinSupply",
    color='Class'
).interactive()

alt.Chart(...)